In [4]:
import boto3
import io
import re
import numpy as np
import os
import xml.etree.ElementTree as ET 
from google.cloud import vision
import requests
from time import time
from utilities.score_computors import jaccard_similarity
#from utilities.pythonDB import writeToDB, recordsExists

In [5]:
def extract_clean_str(original_str):
    original_str = original_str.replace('<br/>', '')
    return re.sub('[^a-zA-Z0-9]+', ' ', original_str, flags=re.UNICODE).lower().strip().replace(' ', '')

def get_mapped(path):
    key_map, dict_idx = {}, 0
    for r, _, f in os.walk(dir_path):
        for file in f:
            if '.JPG' in file or '.jpg' in file:
                key_map[dict_idx] = file
                dict_idx += 1
    idxs = list(key_map.keys())          
    shuffled_idxs = np.random.randint(0, len(idxs), len(idxs))
    return key_map, shuffled_idxs

def fetch_from_xml_natocr(path, file):
    tree = ET.parse(os.path.join(path, file)).getroot()  
    return extract_clean_str(''.join([e[0].text for e in tree.iter(tag='object')]))

In [6]:
dir_path = 'datasets/neocr_dataset/noises'
xml_path = '/'.join(dir_path.split('/')[:2])+'/OCR'
dict_files, files_idx = get_mapped(dir_path)

In [67]:
file = dict_files.get(files_idx[10])
file_name = '_'.join(file.split('_')[-2:])
xml_file_name = file_name[:len(file_name)-4] + '.xml' 
ground_truth_str = fetch_from_xml_natocr(xml_path, xml_file_name)

In [80]:
file = dict_files.get(files_idx[0])

In [82]:
with io.open(os.path.join(dir_path, file), 'rb') as image_file:
    content = (image_file.read())

In [84]:
imgobj = {'Bytes': content}
client=boto3.client('rekognition', region_name='us-east-1')
response=client.detect_text(Image=imgobj)

In [85]:
detected_str = extract_clean_str(''.join([txt['DetectedText'] for txt in response['TextDetections'] \
                                                   if txt['Type']=='WORD']))

In [88]:
file_name = '_'.join(file.split('_')[-2:])
xml_file_name = file_name[:len(file_name)-4] + '.xml' 
ground_truth_str = fetch_from_xml_natocr(xml_path, xml_file_name)

In [86]:
detected_str

'miehin2007forthefirsttime50liviongofpeopleonearthalinhistoryincitieglobalcitiesatatemodernexhibitionfiandsecurtieyfreeadmisshangnal43mumbal42'

In [89]:
bground_truth_str

'in2007forthefirsttimeinhistory50ofpeopleoneartharelivingincitiesglobalcitiesatatemodernexhibitionlandsecuritiesderwentlondonfreeadmissshanghai31mumbai42'

In [7]:
dict_files.get(0)

'add_impulse_noise_4_img_297611753.jpg'

In [ ]:
000000365822.jpg-image_labeling-gc

In [1]:
from utilities.pythonDB import writeToDB_label, recordsExists

In [4]:
recordsExists('000000365822.jpg', 'image_labeling', 'gc')

False

In [5]:
count_query = "select count(*) from research_exploration WHERE file_name='{}' and dataset='{}' \
    and label='{}'".format('000000365822.jpg', 'image_labeling', 'gc')
count_query

"select count(*) from research_exploration WHERE file_name='000000365822.jpg' and dataset='image_labeling'     and label='gc'"